In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/santiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.metrics import jaccard_distance
from scipy.stats import pearsonr
input_file = 'trial/STS.input.txt'
with open(input_file) as f:
    input_data = f.readlines()
distances = []
for i in input_data:
    sentences = nltk.sent_tokenize(i[4:])
    words = [nltk.word_tokenize(sent) for sent in sentences]
    distances.append(jaccard_distance(set(words[0]),set(words[1])))
print(distances)

[0.6923076923076923, 0.7368421052631579, 0.6666666666666666, 0.5454545454545454, 0.7692307692307693, 0.8620689655172413]


In [4]:
gold_file = 'trial/STS.gs.txt'
with open(gold_file) as f:
    gold_data = f.readlines()
gold = [int(g[4:5]) for g in gold_data]
pearsonr(distances, gold)[0]

0.4143770872333895

In [19]:
# Loads the different training data into a list 
langs = ['deu','eng','fra','ita','nld','spa']
training = []
for i in langs: 
    file='langId/'+i+'_trn.txt'
    with open(file) as f:
         training.append(f.readlines())

In [7]:
# Loads the different testing data into a list
test = []
for i in langs: 
    file='langId/'+i+'_tst.txt'
    with open(file) as f:
         test.append(f.readlines())

In [31]:
# Removes punctuations and sets all text to lower case and removes continuous
# white spaces for training data
stdTraining = []
for i in training:
    langtrn = [] 
    for c in i:                
        words = nltk.word_tokenize(c) 
        words = [word.lower() for word in words if word.isalpha()]
        words = [" ".join(word.split()) for word in words]
        langtrn.append(words)
    stdTraining.append(langtrn)

In [42]:
# Removes punctuations and sets all text to lower case and removes continuous
# white spaces for testing data
stdTest = []
for i in test:
    langtst = [] 
    for c in i:                
        words = nltk.word_tokenize(c)                             
        words = [word.lower() for word in words if word.isalpha()]
        langtst.append(words) 
    stdTest.append(langtst)

In [32]:
# Concats all sentences with a double space between them for each lang in the training
concatLangTrain = []
for i in stdTraining: 
    langSentences = ""
    for s in i:
        sentence = ""
        for w in s:
            sentence = sentence + " " + w
        sentence = sentence[1:]
        langSentences = langSentences + "  " + sentence
    langSentences = langSentences[2:]
    concatLangTrain.append(langSentences)

In [58]:
concatLangTest = []
for i in stdTest: 
    #langSentences = ""
    langSentences = []
    for s in i:
        sentence = ""
        for w in s:
            sentence = sentence + " " + w
        sentence = sentence[1:]
        #langSentences = langSentences + "  " + sentence
        langSentences.append(sentence)
    #langSentences = langSentences[2:]
    concatLangTest.append(langSentences)

In [73]:
from nltk.collocations import TrigramCollocationFinder
trigramTrain = []
for i in concatLangTrain:
    trainFinder = TrigramCollocationFinder.from_words(i)
    trainFinder.apply_freq_filter(1000)
    trigramTrain.append([tr for tr in trainFinder.ngram_fd.items()])
    

In [67]:
trigramTest = []
for i in concatLangTest:
    langTrigram = []
    for c in i:
        testFinder = TrigramCollocationFinder.from_words(c)
        #testFinder.apply_freq_filter(3)
        langTrigram.append([tr for tr in testFinder.ngram_fd.items()])
    trigramTest.append(langTrigram)
#for i in concatLangTest:
#    testFinder = TrigramCollocationFinder.from_words(i)
#    testFinder.apply_freq_filter(5)
#    trigramTest.append([tr for tr in testFinder.ngram_fd.items()])

In [63]:
testSorted = []
for i in trigramTest:
    sortSent = []
    for c in i:
        sortSent.append(sorted(c, key=lambda x:x[1], reverse=True))
    #testSorted.append(sorted(i, key=lambda x: x[1], reverse=True))
    testSorted.append(sortSent)
trainSorted = []
for i in trigramTrain:
    trainSorted.append(sorted(i, key=lambda x: x[1], reverse=True))

In [74]:
from nltk.metrics import jaccard_distance
distances = []
for i in trigramTest:
    distancesLang = []
    for s in i:
        distancesSentence = []
        for c in trigramTrain:
            distancesSentence.append(jaccard_distance(set([x[0] for x in s]),set([x[0] for x in c])))
        #distancesLang.append(jaccard_distance(set([x[0] for x in i]),set([x[0] for x in c])))
        distancesLang.append(distancesSentence)
    distances.append(distancesLang)
#distances

In [75]:
from nltk.metrics.scores import accuracy
from nltk.metrics import ConfusionMatrix

#results = []
#for i in distances:
#    results.append(langs[i.index(min(i))])

#print('accuracy: ' + str(accuracy(langs, results)))
#cm = ConfusionMatrix(langs, results)
refs = []
count = 0
results = []
for i in distances:
    for c in i:
        results.append(langs[c.index(min(c))])
        refs.append(langs[count])
    count +=1
print('accuracy: ' + str(accuracy(refs, results)))
cm = ConfusionMatrix(refs, results)
print(cm.pretty_format())

accuracy: 0.9867282458275672
    |    d    e    f    i    n    s |
    |    e    n    r    t    l    p |
    |    u    g    a    a    d    a |
----+-------------------------------+
deu |<9905>   1    6    .   75    3 |
eng |   33<9667>  73    5  205    4 |
fra |   18    7<9948>   7   16    4 |
ita |    4    4   62<9887>  10   33 |
nld |   30    5   13    6<9946>   . |
spa |    4    9  104   51    4<9828>|
----+-------------------------------+
(row = reference; col = test)

